# Understanding Student Data - Analysing and visualizing data from a CSV file

In this module, we will be working on a project that involves analyzing and visualizing data from a CSV file. The specific data set we will be using is the "Student Performance Data Set" from https://archive.ics.uci.edu/ml/datasets/Student+Performance.

You will learn about:

- Loading CSVs
- PyArrow
- Summary Stats
- Correlations
- Cross Tabulations
- Visualizations


## Loading from a ZIP File

In [1]:
import pandas as pd

import io
from urllib.request import urlopen
import zipfile

In [2]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip'

fin = urlopen(url)
with zipfile.ZipFile(io.BytesIO(fin.read())) as zip:
  print(zip.namelist())

['student-mat.csv', 'student-por.csv', 'student-merge.R', 'student.txt']


In [3]:
%%bash
mkdir data
curl --output data/student.zip https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip 

mkdir: cannot create directory ‘data’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 20478    0 20478    0     0  25987      0 --:--:-- --:--:-- --:--:-- 25954


In [4]:
path = 'data/student.zip'
with zipfile.ZipFile(path) as zip:
  print(zip.namelist())

['student-mat.csv', 'student-por.csv', 'student-merge.R', 'student.txt']


In [5]:
with zipfile.ZipFile(path) as zip:
  df = pd.read_csv(zip.open('student-mat.csv'), sep=';')

In [6]:
df

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,5,5,4,4,5,4,11,9,9,9
391,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,3,14,16,16
392,MS,M,21,R,GT3,T,1,1,other,other,...,5,5,3,3,3,3,3,10,8,7
393,MS,M,18,R,LE3,T,3,2,services,other,...,4,4,1,3,4,5,0,11,12,10


In [7]:
df.memory_usage(deep=True).sum()

454598

In [8]:
path = 'data/student.zip'
with zipfile.ZipFile(path) as zip:
  df = pd.read_csv(zip.open('student-mat.csv'), sep=';',
                  dtype_backend='pyarrow', engine='pyarrow')

In [9]:
df.memory_usage(deep=True).sum()

98703

In [10]:
%%timeit
# pandas 2 optimizations
# dtype_backend - store with pyarrow
# engine - multithreaded w/ pyarrow
path = 'data/student.zip'

with zipfile.ZipFile(path) as zip:
  df = pd.read_csv(zip.open('student-mat.csv'), sep=';',
                  dtype_backend='pyarrow', engine='pyarrow')

13.8 ms ± 6.28 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
%%timeit
# pandas 2 optimizations
# dtype_backend - store with pyarrow
# engine - multithreaded w/ pyarrow
path = 'data/student.zip'

with zipfile.ZipFile(path) as zip:
  df = pd.read_csv(zip.open('student-mat.csv'), sep=';')

5.76 ms ± 1.98 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
pd.read_csv?

Signature:
pd.read_csv(
    filepath_or_buffer: 'FilePath | ReadCsvBuffer[bytes] | ReadCsvBuffer[str]',
    *,
    sep: 'str | None | lib.NoDefault' = <no_default>,
    delimiter: 'str | None | lib.NoDefault' = None,
    header: "int | Sequence[int] | None | Literal['infer']" = 'infer',
    names: 'Sequence[Hashable] | None | lib.NoDefault' = <no_default>,
    index_col: 'IndexLabel | Literal[False] | None' = None,
    usecols: 'list[HashableT] | Callable[[Hashable], bool] | None' = None,
    dtype: 'DtypeArg | None' = None,
    engine: 'CSVEngine | None' = None,
    converters: 'Mapping[Hashable, Callable] | None' = None,
    true_values: 'list | None' = None,
    false_values: 'list | None' = None,
    skipinitialspace: 'bool' = False,
    skiprows: 'list[int] | int | Callable[[Hashable], bool] | None' = None,
    skipfooter: 'int' = 0,
    nrows: 'int | None' = None,
    na_values: 'Sequence[str] | Mapping[str, Sequence[str]] | None' = None,
    keep_default_na: 'bool' = True,
    n

## Summary Stats



### Attributes for both student-mat.csv (Math course) and student-por.csv (Portuguese language course) datasets:
1. school - student's school (binary: 'GP' - Gabriel Pereira or 'MS' - Mousinho da Silveira)
2. sex - student's sex (binary: 'F' - female or 'M' - male)
3. age - student's age (numeric: from 15 to 22)
4. address - student's home address type (binary: 'U' - urban or 'R' - rural)
5. famsize - family size (binary: 'LE3' - less or equal to 3 or 'GT3' - greater than 3)
6. Pstatus - parent's cohabitation status (binary: 'T' - living together or 'A' - apart)
7. Medu - mother's education (numeric: 0 - none, 1 - primary education (4th grade), 2 â€“ 5th to 9th grade, 3 â€“ secondary education or 4 â€“ higher education)
8. Fedu - father's education (numeric: 0 - none, 1 - primary education (4th grade), 2 â€“ 5th to 9th grade, 3 â€“ secondary education or 4 â€“ higher education)
9. Mjob - mother's job (nominal: 'teacher', 'health' care related, civil 'services' (e.g. administrative or police), 'at_home' or 'other')
10. Fjob - father's job (nominal: 'teacher', 'health' care related, civil 'services' (e.g. administrative or police), 'at_home' or 'other')
11. reason - reason to choose this school (nominal: close to 'home', school 'reputation', 'course' preference or 'other')
12. guardian - student's guardian (nominal: 'mother', 'father' or 'other')
13. traveltime - home to school travel time (numeric: 1 - <15 min., 2 - 15 to 30 min., 3 - 30 min. to 1 hour, or 4 - >1 hour)
14. studytime - weekly study time (numeric: 1 - <2 hours, 2 - 2 to 5 hours, 3 - 5 to 10 hours, or 4 - >10 hours)
15. failures - number of past class failures (numeric: n if 1<=n<3, else 4)
16. schoolsup - extra educational support (binary: yes or no)
17. famsup - family educational support (binary: yes or no)
18. paid - extra paid classes within the course subject (Math or Portuguese) (binary: yes or no)
19. activities - extra-curricular activities (binary: yes or no)
20. nursery - attended nursery school (binary: yes or no)
21. higher - wants to take higher education (binary: yes or no)
22. internet - Internet access at home (binary: yes or no)
23. romantic - with a romantic relationship (binary: yes or no)
24. famrel - quality of family relationships (numeric: from 1 - very bad to 5 - excellent)
25. freetime - free time after school (numeric: from 1 - very low to 5 - very high)
26. goout - going out with friends (numeric: from 1 - very low to 5 - very high)
27. Dalc - workday alcohol consumption (numeric: from 1 - very low to 5 - very high)
28. Walc - weekend alcohol consumption (numeric: from 1 - very low to 5 - very high)
29. health - current health status (numeric: from 1 - very bad to 5 - very good)
30. absences - number of school absences (numeric: from 0 to 93)

### these grades are related with the course subject, Math or Portuguese:
31. G1 - first period grade (numeric: from 0 to 20)
31. G2 - second period grade (numeric: from 0 to 20)
32. G3 - final grade (numeric: from 0 to 20, output target)

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
df.quantile(.99)

In [ ]:
df.quantile(.99, numeric_only=True)

In [ ]:
df.quantile([.01,.3,.5, .8, .99], numeric_only=True)

## Correlations

In [ ]:
df.corr(numeric_only=True)

In [ ]:
(df
 .corr(numeric_only=True)
 .style
 .background_gradient()
)

In [ ]:
(df
 .corr(numeric_only=True)
 .style
 .background_gradient(cmap='RdBu')
)

In [ ]:
(df
 .corr(numeric_only=True)
 .style
 .background_gradient(cmap='RdBu', vmin=-1, vmax=1)
)

In [ ]:
(df
 .corr(numeric_only=True, method='spearman')
 .style
 .background_gradient(cmap='RdBu', vmin=-1, vmax=1)
)

## Categorical Columns

In [ ]:
# pandas 1.x
df.select_dtypes(object)

In [ ]:
# pandas 2 w/ pyarrow
df.select_dtypes('string')

In [ ]:
# pandas 2 w/ pyarrow
df.select_dtypes('string[pyarrow]')

In [ ]:
df.famsize.value_counts()

In [ ]:
df.higher.value_counts()

In [ ]:
pd.crosstab(df.sex, df.higher)

In [ ]:
# normalize across all cells
pd.crosstab(df.sex, df.higher, normalize=True)

In [ ]:
(pd.crosstab(df.sex, df.higher, normalize=True)
 .style
 .format('{:.2%}')
)

In [ ]:
(pd.crosstab(df.sex, df.higher, normalize='index')
 .style
 .format('{:.2%}')
)

In [ ]:
# normalize down columns
(pd.crosstab(df.sex, df.higher, normalize='columns')
 .style
 .format('{:.2%}')
)

## Visualizations

In [ ]:
# health - current health status (numeric: from 1 - very bad to 5 - very good)
df.health.hist(figsize=(8,3), bins=5)

In [ ]:
# G3 - final grade (numeric: from 0 to 20, output target)
df.G3.hist(figsize=(8,3), bins=20)

In [ ]:
df.plot.scatter(x='Medu', y='G3', figsize=(8,3))

In [ ]:
# Medu - mother's education (numeric: 0 - none, 1 - primary education (4th grade), 
# 2 - 5th to 9th grade, 3 - secondary education or 4 - higher education)

df.plot.scatter(x='Medu', y='G3', alpha=.1, figsize=(8,3))

In [ ]:
import numpy as np
amt = .5
(df
 .assign(edu_jit=df.Medu + np.random.random(len(df))*amt - amt/2)
 .plot.scatter(x='edu_jit', y='G3', alpha=.3, figsize=(8,3))
)

In [ ]:
import numpy as np
def jitter(df, col, amt=.5):
    return df[col] + np.random.random(len(df))*amt - amt/2

(df
 .assign(edu_jit=lambda df_: jitter(df_, col='Medu'))
 .plot.scatter(x='edu_jit', y='G3', alpha=.3, figsize=(8,3))
)

In [ ]:
# add jitter to a helpers.py file

In [ ]:
# guardian - student's guardian (nominal: 'mother', 'father' or 'other')
df.guardian.value_counts().plot.bar(figsize=(8,3))

In [ ]:
df.guardian.value_counts().plot.barh(figsize=(8,3))